# Delays estimation

In this notebook, we build the models for the delays of the different mean of transport. The delay between two stops for a given trip is modelled by an exponential random variable. Note that this distribution only models positive values. Therefore we consider that trains/buses/etc either arrive on time or late.

## Setup PySpark

In [1]:
%load_ext sparkmagic.magics

In [2]:
import os
import warnings
import pandas as pd
warnings.simplefilter(action='ignore', category=UserWarning)

username = os.environ['RENKU_USERNAME']
print(username)

verardo


In [3]:
server = "http://iccluster029.iccluster.epfl.ch:8998"
from IPython import get_ipython
get_ipython().run_cell_magic('spark', line="config", 
                             cell="""{{ "name":"{0}-aces", "executorMemory":"20G", "executorCores":6, "numExecutors":10 }}""".format(username))

In [4]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-aces -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9408,application_1652960972356_5234,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


## Data for transport in the vicinity of Zürich
As our planner is limited to a circular area of 15km radius around Zürich, we keep only the data corresponding to the corresponding stations.

In [5]:
%%spark
istdaten = spark.read.orc('/data/sbb/orc/istdaten/').cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
%%spark

# Get the nodes that are around Zürich
nodes_zh_id = spark.read.orc('/group/aces/nodes.orc').select('stop_id').rdd.flatMap(lambda x: x).map(lambda x: x.split(':')[0]).distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
%%spark
istdaten_zh = istdaten.filter(istdaten.bpuic.isin(nodes_zh_id))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Preprocessing

We preprocess the SBB data in the following way :

- Filter out additional trips that are not part of the regular schedule (zusatzfahrt_tf == False)
- Filter out trips that did not arrive at destination (faellt_aus_tf == False)
- Keep only stops that have a effective (REAL), estimated (GESCHAETZT) or forecasted (PROGNOSE) arrival and departure times
- Filter out stops that have no arrival time (an_prognose != '')
- Filter out stops that have no departure time (ankunftszeit != '')
- Filter out rows where the train/bus/etc did not stop, we only represent direct edges (durchfahrt_tf == False)
- Filter out stops that are not during the week days and not during working hours (departure after 0600 and arrival before 2000)

In [8]:
%%spark
from pyspark.sql.functions import hour, col, to_timestamp, when, dayofweek

istdaten_zh_cleaned = istdaten_zh.filter((col('zusatzfahrt_tf') == False)
                                         & (col('faellt_aus_tf') == False)
                                         & (col('an_prognose_status').isin(['REAL', 'PROGNOSE', 'GESCHAETZT']))
                                         & (col('ab_prognose_status').isin(['REAL', 'PROGNOSE', 'GESCHAETZT']))
                                         & (col('an_prognose') != '')
                                         & (col('ankunftszeit') != '')
                                         & (col('durchfahrt_tf') == False)
                                         & (dayofweek(to_timestamp(col('an_prognose'), format="dd.MM.yyyy HH:mm:ss")) % 7 > 1) # To keep only week days
                                         & (hour(to_timestamp(col('ab_prognose'), format="dd.MM.yyyy HH:mm:ss")) >= 6)
                                         & (hour(to_timestamp(col('an_prognose'), format="dd.MM.yyyy HH:mm:ss")) <= 20)).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Remove special transports

The data contains rows for more special transports such as agency trains (AG), special trains (EXT), express bus (EXB), Voralpen-Express (VAE) and low-floor bus (NFB). We remove them as we concentrate on the Zürich area where the user is unlikely to take such transports.

In [9]:
%%spark
istdaten_zh_cleaned.select('verkehrsmittel_text').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(verkehrsmittel_text=u'IC'), Row(verkehrsmittel_text=u'ZUG'), Row(verkehrsmittel_text=u'RE'), Row(verkehrsmittel_text=u'NJ'), Row(verkehrsmittel_text=u'T'), Row(verkehrsmittel_text=u'B'), Row(verkehrsmittel_text=u'EC'), Row(verkehrsmittel_text=u'IR'), Row(verkehrsmittel_text=u'Trm'), Row(verkehrsmittel_text=u'ICE'), Row(verkehrsmittel_text=u'NFB'), Row(verkehrsmittel_text=u'S'), Row(verkehrsmittel_text=u'FUN'), Row(verkehrsmittel_text=u'Bus'), Row(verkehrsmittel_text=u'EXT'), Row(verkehrsmittel_text=u'RJX'), Row(verkehrsmittel_text=u'RJ'), Row(verkehrsmittel_text=u'')]

In [10]:
%%spark

# Filter out special kind of trains/transport to avoid bias the delays
unwanted_transport = ['AG', 'EXT', 'EXB', 'VAE', 'NFB']
istdaten_zh_cleaned = istdaten_zh_cleaned.filter(~col('verkehrsmittel_text').isin(unwanted_transport))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Aggregate the different types of transports

The produkt_id column contains different types of transports such as "IC", "IR", "ZUG" that we all aggregate to "Train" to then compute the delays for each of these aggregate categories, depending on the hour of the day.

In [11]:
%%spark
istdaten_zh_cleaned.select('produkt_id').distinct().collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(produkt_id=u'Standseilbahn'), Row(produkt_id=u'Tram'), Row(produkt_id=u'Zug'), Row(produkt_id=u'Bus'), Row(produkt_id=u'')]

In [12]:
%%spark
import pyspark.sql.functions as F
from pyspark.sql.functions import col

# Translate the transport meaning
transport_mapping = {
    "BUS": "Bus",
    "Zug": "Train",
    "Bus": "Bus",
    "BUS": "Bus",
    "ICE": "Train",
    "IC": "Train",
    "IR": "Train",
    "ZUG": "Train",
    "RE": "Train",
    "S": "Train",
    "R": "Train",
    "PE": "Train",
    "EC": "Train",
    "NJ": "Train",
    "RJ": "Train",
    "RJX": "Train",
    "FUN": "Other",
    "Standseilbahn": "Other",
    "B": "Bus",
    "T": "Tram",
    "Trm": "Tram",
    "Tram": "Tram",
    "TGV": "Train",
    "": "Other"
}

@F.udf
def map_transport(x):
    return transport_mapping[x]

istdaten_zh_cleaned = istdaten_zh_cleaned.withColumn('produkt_id', when(col('produkt_id') == '', map_transport(col('verkehrsmittel_text'))).otherwise(map_transport(col('produkt_id'))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Delay at each stop
Now, we compute the delay for each of the row we have at disposition. It is expressed in seconds and defined as :

$$
\text{\{expected hour of arrival at the stop (according to the timetable)\} } - \text{\{actual arrival at the stops according to the measurement\}}
$$

The train/bus might arrive in advance at the stop, in which case we clip its delay to 0 instead of keeping a negative value. This is due to our modelling of the delay by an exponential distribution which can only take positive values.

In [13]:
%%spark
# Compute the delay, clipping to 0 when it is negative
istdaten_zh_cleaned = istdaten_zh_cleaned.withColumn('delay', to_timestamp(istdaten_zh_cleaned.an_prognose, 'dd.MM.yyyy HH:mm:ss').cast('long') - to_timestamp(istdaten_zh_cleaned.ankunftszeit, 'dd.MM.yyyy HH:mm').cast('long'))
#istdaten_zh_cleaned = istdaten_zh_cleaned.withColumn('delay', when(col('delay') < 0, 0).otherwise(col('delay')))
istdaten_zh_cleaned = istdaten_zh_cleaned.withColumn('expected_arrival_hour', hour(to_timestamp(istdaten_zh_cleaned.ankunftszeit, 'dd.MM.yyyy HH:mm')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Aggregated delay

We then aggregate the delays w.r.t. different categories :

- by stop id : different stops may have different statistics regarding the delays of the transports going through it
- by type of transport (produkt_id) : trains can have different behaviors in terms of delays compared to buses that drive through the city center for example
- by the hour of arrival : we can expect transport to have more or less delays in different time of the day. For example, during the rush hour around 0800, it is likely there will be more delays than during the afternoon at 1500 for example.

We compute the average delays for each combination of these three variables present in the data.

In [23]:
%%spark
istdaten_zh_cleaned_percentile = (istdaten_zh_cleaned.groupBy(['bpuic', 'produkt_id', 'expected_arrival_hour'])
    .agg(
        F.expr('percentile(delay, array(0.95))')[0].alias('95perc'),
        F.expr('percentile(delay, array(0.05))')[0].alias('5perc')
    ))
#istdaten_zh_cleaned = istdaten_zh_cleaned.join(mean_df, (mean_df.bpuic == istdaten_zh_cleaned.bpuic) & (mean_df.produkt_id == istdaten_zh_cleaned.produkt_id) & (istdaten_zh_cleaned.expected_arrival_hour == mean_df.expected_arrival_hour))
#istdaten_zh_cleaned = istdaten_zh_cleaned.join(mean_df, on=['bpuic', 'produkt_id', 'expected_arrival_hour'])
istdaten_zh_cleaned = (istdaten_zh_cleaned.join(istdaten_zh_cleaned_percentile, on=["bpuic", "produkt_id","expected_arrival_hour"])
                           .filter((col("delay")>=col("5perc"))&(col("delay")<=col("95perc")))
                      )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
%%spark
istdaten_zh_cleaned = (istdaten_zh_cleaned.groupBy(['bpuic', 'produkt_id', 'expected_arrival_hour'])
    .agg(
        F.expr('mean(delay)').alias('mean_delay'),
        F.expr('std(delay)').alias('std_delay'),
        F.expr('percentile(delay,array(0.5))')[0].alias('median_delay')
    )).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
%%spark
istdaten_zh_cleaned.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [19]:
%%spark
istdaten_zh_cleaned.distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [28]:
%%spark
istdaten_zh_cleaned.select(
    istdaten_zh_cleaned.bpuic.alias('end_bpuic'),
    istdaten_zh_cleaned.produkt_id.alias('transport'),
    istdaten_zh_cleaned.mean_delay,
    istdaten_zh_cleaned.std_delay,
    istdaten_zh_cleaned.median_delay,
    istdaten_zh_cleaned.expected_arrival_hour.alias('hour')
).write.save('/group/aces/istdaten_delays.orc', format='orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…